## Tweepy API Scrape 
_Authors: Yichen Hu & Christopher Thompson_

### Imports ###

In [2]:
import tweepy
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import json

#pip install may be required for some libraries 

### GeoPy Functions / Code ###

In [3]:
geolocator = Nominatim(user_agent = "DSI_US_8_LA")

In [4]:
# Function to take a user-entered address and return it's coordinates
def latlong(point):
    coord = geolocator.geocode(point)
    return (coord.latitude,coord.longitude)

In [5]:
cord = latlong('360 East 2nd Street, Los Angeles, CA')

### Twitter Authentication / Tweepy Initialization ###

In [6]:
json_file = open('creds/twitter_creds.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [7]:
#Code taken from http://www.dealingdata.net/2016/07/23/PoGo-Series-Tweepy/
auth = tweepy.OAuthHandler(json_data['API key'],json_data['API secret key'])
auth.set_access_token(json_data['Access token'], json_data['Access token secret'])

#Creating a twitter API wrapper using tweepy
api = tweepy.API(auth)

#Error handling
if (not api):
    print ("Problem connecting to API")

In [8]:
#Function to return the hashtags in a tweet
def hashtag_process(raw):
    if len(raw)==0:
        return None
    else:
        output = []
        for x in raw:
            if x.get('text') != 'LAtraffic':
                output.append(x.get('text'))
        return output

TweetSearch Function Parameters

- location - exact latitude and longitude to start search from, should be the output of latlong function
- distance - distance in miles to search for (default is 25)
- user - twitter hashtag to pull Tweets from (default is TotalTrafficLA)
- limit - number of tweets to initially pull from the user before applying location filtering (default is 100)

In [9]:
def TweetSearch(location,distance = 25,user = 'TotalTrafficLA',limit = 100):
    full_text = []
    author = []
    creation_time = []
    hashtags = []
    geo = []
    
    for tweet in tweepy.Cursor(api.user_timeline,id=user,tweet_mode='extended').items(limit):
        if tweet.geo != None:
            if geodesic(location,tweet.geo['coordinates']).miles <= distance: #only allow tweets from within the location radius
                #parse out tweet data and save to lists
                full_text.append(tweet.full_text)
                author.append(tweet.author.screen_name)
                creation_time.append(tweet.created_at)
                hashtags.append(hashtag_process(tweet.entities['hashtags'])) #run function to return a tweet's hashbags in a good form
                geo.append(tweet.geo['coordinates'])
    
    return pd.DataFrame([full_text,author,geo,hashtags,creation_time],\
                        index=['Tweet Text','Author','Coordinates','Hashtags (Location)','Creation Time (UTC)']).T

In [10]:
#Pass in latitude and longitude points from Flask 
df = TweetSearch("34.1581656060606, -118.333947181818")

In [11]:
df

Tweet Text          Author  \
0   A crash is blocking the carpool lane. in #Arca...  TotalTrafficLA   
1   A crash is blocking the carpool lane. in #Arca...  TotalTrafficLA   
2   A motorcycle crash was moved to the right shou...  TotalTrafficLA   
3   A motorcycle crash is blocking the left lane. ...  TotalTrafficLA   
4   A crash was moved to the right shoulder. in #W...  TotalTrafficLA   
5   A motorcycle crash is blocking the left lane. ...  TotalTrafficLA   
6   A crash was moved to the right shoulder. in #W...  TotalTrafficLA   
7   Accident, left lane blocked in #WoodlandHills ...  TotalTrafficLA   
8   A stalled car was cleared from the left lane. ...  TotalTrafficLA   
9   Crash blocking the trans to the 110 north in #...  TotalTrafficLA   
10  Crash blocking the trans to the 110 in #Elysia...  TotalTrafficLA   
11  A crash is blocking the two left lanes in #LaC...  TotalTrafficLA   
12  !!sigalert!! police activity has all lanes blo...  TotalTrafficLA   
13  !! sigalert !! a big rig tanker is stalled in ...  TotalTrafficLA   
14  !! sigalert !! a crash is blocking the two lef...  TotalTrafficLA   
15  Accident cleared in #SouthLa on 110 SB approac...  TotalTrafficLA   
16  Accident cleared in #SouthLa on 110 SB approac...  TotalTrafficLA   
17  !! sigalert !! the left lane is blocked due to...  TotalTrafficLA   
18  A crash involving a flipped over car is blocki...  TotalTrafficLA   
19  A crash involving a flipped over car is blocki...  TotalTrafficLA   
20  Accident, left lane blocked in #ElysianPark on...  TotalTrafficLA   
21  A crash is on the right shoulder. chphas the r...  TotalTrafficLA   
22  !!sigalert cleared!! solo crash cleared from t...  TotalTrafficLA   
23  !!sigalert!! solo crash blocking left lane. in...  TotalTrafficLA   
24  Accident cleared in #Commerce on 5 SB at Garfi...  TotalTrafficLA   
25  Solo crash blocking lanes. in #SantaClarita on...  TotalTrafficLA   
26  Crash blocking lanes of off-ramp. in #Commerce...  TotalTrafficLA   
27  !! sigalert !! a crash is blocking the left la...  TotalTrafficLA   
28  !! sigalert !! a crash is blocking the two rig...  TotalTrafficLA   
29  A crash is blocking the two right lanes. in #W...  TotalTrafficLA   
..                                                ...             ...   
31  !! sigalert !! a crash is blocking the two rig...  TotalTrafficLA   
32  Accident, right lane blocked in #Downey on 5 N...  TotalTrafficLA   
33  The transition to the 405 north is closed due ...  TotalTrafficLA   
34  !! sigalert !! a crash has only the right lane...  TotalTrafficLA   
35  !! sigalert !! a crash is blocking the three l...  TotalTrafficLA   
36  A stalled car has all lanes temporarily stoppe...  TotalTrafficLA   
37  A motorcycle crash is blocking the left lane. ...  TotalTrafficLA   
38  A motorcycle crash is blocking the left lane. ...  TotalTrafficLA   
39  Accident, two lanes blocked in #Irwindale on 2...  TotalTrafficLA   
40  Reopened in #MalibuPacificPalisades on Hwy 27 ...  TotalTrafficLA   
41  Crash has the carpool and left lane blocked. i...  TotalTrafficLA   
42  Crash is blocking the carpool &amp; left lane ...  TotalTrafficLA   
43  !!closed!!  all lanes shutdown due to a brush ...  TotalTrafficLA   
44  Crash has the two right lanes lanes. in #South...  TotalTrafficLA   
45  Crash has the two right lanes lanes. in #South...  TotalTrafficLA   
46  Brush fire has all lanes blocked. in #Woodland...  TotalTrafficLA   
47  !!cleared sigalert!!  earlier crash moved out ...  TotalTrafficLA   
48  !!cleared sigalert!!  earlier crash moved out ...  TotalTrafficLA   
49  Earlier stalled car moved out of lanes in #Dow...  TotalTrafficLA   
50  !!sigalert!!  car fire blocking the carpool an...  TotalTrafficLA   
51  Car fire blocking the carpool and left lane in...  TotalTrafficLA   
52  Stalled car blocking the right lane in #Downto...  TotalTrafficLA   
53  Car fire blocking the carpool lane in #Mission...  TotalTrafficLA   
54  Earlier stalled car move

In [ ]:
df.to_csv('data/first_scrape.csv',index=False)